# Main

> FrankenUI Tasks Example

In [2]:
#| default_exp main

## Imports

In [3]:
#| export
from fasthtml.common import *
import fasthtml.common as fh
from fh_frankenui.components import *
from fh_frankenui.core import *
import re
from fasthtml.components import Uk_theme_switcher
from utils import hjs

In [4]:
#| hide
#| eval: false
from utils import create_server
from fasthtml.jupyter import *

In [5]:
#| export
app,rt = fast_app(pico=False, hdrs=(*Theme.blue.headers(),*hjs))

In [7]:
#| hide
#| eval: false
server, Show = create_server(app)

In [8]:
#| export
def with_layout(func):
    def wrapper():
        original_content = func()
        return Body(cls="bg-background text-foreground")(
            Div(cls="flex flex-col md:flex-row w-full")(
                Button(UkIcon("menu",50,50,cls='mt-4'), cls="md:hidden mb-4", uk_toggle="target: #mobile-sidebar"),
                Div(sidebar, id='mobile-sidebar', hidden=True),
                
                Div(cls="md:flex w-full")(
                    Div(sidebar, cls="hidden md:block w-1/5"),
                    Div(original_content, cls='md:w-4/5 w-full', id="mobile-sidebar")
                )
            ),
        )
    wrapper.__name__ = func.__name__
    return wrapper

## Examples

In [9]:
#| export
from tasks import tasks_homepage
from cards import cards_homepage
from dashboard import dashboard_homepage
from forms import forms_homepage
from music import music_homepage
from auth import auth_homepage
from playground import playground_homepage
from mail import mail_homepage

In [10]:
#| export
@rt
@with_layout
def tasks():      return tasks_homepage
@rt
@with_layout
def cards():      return cards_homepage
@rt
@with_layout
def dashboard():  return dashboard_homepage
@rt 
@with_layout
def forms():      return forms_homepage
@rt
@with_layout
def music():      return music_homepage
@rt
@with_layout
def auth():       return auth_homepage
@rt
@with_layout
def playground(): return playground_homepage
@rt
@with_layout
def mail():       return mail_homepage

## Getting Started

In [11]:
#| export
@rt
@with_layout
def getting_started():
    return Container(render_md(open('GettingStarted.md').read()))

## API Reference

In [12]:
#| export
import api_reference

In [ ]:
#| export
reference_fns = L([o for o in dir(api_reference) if o.startswith('docs_')])

In [13]:
reference_fns[:3]

(#3) ['docs_button_link','docs_cards','docs_containers']

In [14]:
#| export
api_ref_rts = [(f"/{o}", rt(f"/{o}")(with_layout(getattr(api_reference, o)))) for o in reference_fns]

In [105]:
#| export
def fnname2title(ref_fn_name): return ref_fn_name[5:].replace('_',' | ').title() 

## Tutorials

## Theme

In [107]:
#| export
@rt
@with_layout
def themeswitcher(): return Div(Uk_theme_switcher(),cls="p-12")

## Page Layout

In [108]:
#| export
sidebar = NavContainer(
     Li(A("Getting Started", href=getting_started)),
     NavParentLi(
         A(FullySpacedDiv("API Reference")),
         NavContainer(
             *[Li(A(fnname2title(o), href=f"/{o}")) for o in reference_fns],
             parent=False),
         ),
     NavParentLi(
        A(FullySpacedDiv('Examples')),
        NavContainer(
            Li(A('Tasks',     href=tasks)),
            Li(A('Cards',     href=cards)),
            Li(A('Dashboard', href=dashboard)),
            Li(A('Form',      href=forms)),
            Li(A('Music',     href=music)),
            Li(A('Auth',      href=auth)),
            Li(A('Playground',href=playground)),
            Li(A('Mail',      href=mail)),
            parent=False),
         ),
      Li(A("Theme",href=themeswitcher)),

    uk_nav=False, cls=(NavT.primary,"space-y-4 p-4 w-full md:w-full"))

In [71]:
#| export
@rt
def index():return getting_started()

In [53]:
#| export
serve()